In [6]:
import numpy as np
import pyaudio
import pygame
import matplotlib.cm as cm
from scipy.fftpack import fft
from math import sin, cos, radians

# Audio parameters
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024
AMPLITUDE_SCALE = 0.01

# Display parameters
WIDTH, HEIGHT = 1600, 900
FPS = 30
BG_COLOR = (10, 10, 20)
PATTERNS = ['circle', 'bars', 'wave', 'spiral']

class AudioVisualizer:
    def __init__(self):
        # Initialize PyAudio
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(
            format=FORMAT,
            channels=CHANNELS,
            rate=RATE,
            input=True,
            frames_per_buffer=CHUNK
        )
        
        # Initialize Pygame
        pygame.init()
        pygame.display.set_caption("Audio Visualizer")
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        self.clock = pygame.time.Clock()
        
        # Visualization state
        self.current_pattern = 0
        self.running = True
        self.color_offset = 0
        
    def get_audio_data(self):
        """Get audio data from microphone and transform to frequency domain"""
        data = np.frombuffer(self.stream.read(CHUNK, exception_on_overflow=False), dtype=np.int16)
        data = data * AMPLITUDE_SCALE
        
        # Calculate FFT and get frequencies magnitude
        fft_data = fft(data)
        freq_data = np.abs(fft_data[0:CHUNK//2]) / (CHUNK//2)
        
        return data, freq_data
    
    def draw_circle_pattern(self, freq_data):
        """Draw a circle pattern with audio reactive spikes"""
        center_x, center_y = WIDTH // 2, HEIGHT // 2
        base_radius = min(WIDTH, HEIGHT) // 4
        
        # Extract bands and normalize them for visualization
        num_bands = 60
        bands = np.array_split(freq_data[:128], num_bands)
        band_averages = [np.mean(band) for band in bands]
        
        for i, magnitude in enumerate(band_averages):
            angle = radians(i * (360 / num_bands))
            
            # Calculate color based on magnitude and offset
            color_val = (i + self.color_offset) % 256
            color = pygame.Color(*[int(x*255) for x in cm.plasma(color_val/255)[:3]])
            
            # Calculate start and end points
            radius = base_radius + magnitude * 150
            x1 = center_x + base_radius * cos(angle)
            y1 = center_y + base_radius * sin(angle)
            x2 = center_x + radius * cos(angle)
            y2 = center_y + radius * sin(angle)
            
            # Draw line
            line_width = max(1, int(magnitude * 5))
            pygame.draw.line(self.screen, color, (x1, y1), (x2, y2), line_width)
    
    def draw_bars_pattern(self, freq_data):
        """Draw a spectrum analyzer style bar pattern"""
        # Number of bars and spacing
        num_bars = 64
        bar_width = WIDTH // num_bars
        
        # Extract and normalize relevant frequency bands
        bands = np.array_split(freq_data[:256], num_bars)
        band_averages = [np.mean(band) for band in bands]
        
        for i, magnitude in enumerate(band_averages):
            # Calculate bar height (proportional to frequency magnitude)
            bar_height = int(magnitude * HEIGHT * 1.5)
            
            # Calculate bar position
            x = i * bar_width
            y = HEIGHT - bar_height
            
            # Color based on frequency
            hue = (i / num_bars * 255 + self.color_offset) % 256
            color = pygame.Color(*[int(x*255) for x in cm.viridis(hue/255)[:3]])
            
            # Draw bar
            pygame.draw.rect(self.screen, color, (x, y, bar_width - 1, bar_height))
    
    def draw_wave_pattern(self, audio_data):
        """Draw a waveform visualization"""
        # Use raw audio data for waveform
        step = len(audio_data) // WIDTH
        points = []
        
        for i in range(0, WIDTH):
            idx = i * step
            if idx < len(audio_data):
                # Calculate y position based on audio amplitude
                y = (HEIGHT // 4) + int(audio_data[idx] * HEIGHT // 8)
                points.append((i, y))
        
        # Draw connecting lines between points with color gradient
        if len(points) > 1:
            for i in range(len(points) - 1):
                p1 = points[i]
                p2 = points[i + 1]
                
                # Color based on position and offset
                hue = (i / WIDTH * 255 + self.color_offset) % 256
                color = pygame.Color(*[int(x*255) for x in cm.coolwarm(hue/255)[:3]])
                
                pygame.draw.line(self.screen, color, p1, p2, 2)
    
    def draw_spiral_pattern(self, freq_data):
        """Draw a spiral pattern reactive to audio"""
        center_x, center_y = WIDTH // 2, HEIGHT // 2
        
        # Extract bass, mid, and treble frequency bands
        bass = np.mean(freq_data[0:10]) * 1.5
        mid = np.mean(freq_data[20:50])
        treble = np.mean(freq_data[50:100])
        
        # Number of spiral arms and base radius
        arms = 5
        max_points = 100
        base_radius = 10 + bass * 100
        
        for arm in range(arms):
            points = []
            arm_offset = arm * (360 / arms)
            
            for i in range(max_points):
                # Calculate spiral parameters
                angle = radians(i * 10 + arm_offset + self.color_offset)
                radius_offset = i * (2 + mid * 5)
                radius = base_radius + radius_offset + (treble * 50 * sin(angle * 3))
                
                # Calculate point position
                x = center_x + radius * cos(angle)
                y = center_y + radius * sin(angle)
                
                if 0 <= x < WIDTH and 0 <= y < HEIGHT:
                    points.append((x, y))
            
            # Draw points with color based on arm and bass intensity
            if points:
                # Create color gradient along the arm
                for i in range(len(points) - 1):
                    p1 = points[i]
                    p2 = points[i + 1]
                    
                    color_val = ((i / max_points) + (arm / arms) + self.color_offset/255) % 1.0
                    color = pygame.Color(*[int(x*255) for x in cm.hsv(color_val)[:3]])
                    
                    # Line width based on bass
                    line_width = max(1, int(3 * bass))
                    pygame.draw.line(self.screen, color, p1, p2, line_width)
    
    def run(self):
        """Main application loop"""
        try:
            while self.running:
                # Handle events
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        self.running = False
                    elif event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_SPACE:
                            # Switch visualization pattern
                            self.current_pattern = (self.current_pattern + 1) % len(PATTERNS)
                        elif event.key == pygame.K_ESCAPE:
                            self.running = False
                
                # Get audio data
                audio_data, freq_data = self.get_audio_data()
                
                # Clear screen
                self.screen.fill(BG_COLOR)
                
                # Draw current pattern
                pattern = PATTERNS[self.current_pattern]
                if pattern == 'circle':
                    self.draw_circle_pattern(freq_data)
                elif pattern == 'bars':
                    self.draw_bars_pattern(freq_data)
                elif pattern == 'wave':
                    self.draw_wave_pattern(audio_data)
                elif pattern == 'spiral':
                    self.draw_spiral_pattern(freq_data)
                
                # Display pattern name
                font = pygame.font.SysFont('Arial', 20)
                pattern_text = font.render(f"Pattern: {pattern.title()} (Space to change)", True, (255, 255, 255))
                self.screen.blit(pattern_text, (10, 10))
                
                instruction_text = font.render("Press ESC to exit", True, (255, 255, 255))
                self.screen.blit(instruction_text, (10, 40))
                
                # Update display
                pygame.display.flip()
                
                # Limit frame rate
                self.clock.tick(FPS)
                
                # Update color offset for animation effect
                self.color_offset = (self.color_offset + 1) % 256
                
        finally:
            # Clean up
            self.stream.stop_stream()
            self.stream.close()
            self.p.terminate()
            pygame.quit()

if __name__ == "__main__":
    visualizer = AudioVisualizer()
    visualizer.run()